In [1]:
import polars as pl
import numpy as np
import time
import pickle


print(pl.__version__)

1.17.1


In [2]:
n_rows = 1_000_000
np.random.seed(100)

data = {
    "id": np.arange(n_rows).tolist(),
    "other_cd": (np.random.choice(list(range(n_rows)), size=n_rows, replace=False)).tolist(),
    "value": (np.random.rand(n_rows) * 100).tolist(),  # random value between 0 and 100
    "category": (np.random.choice(["A", "B", "C", "D"], size=n_rows)).tolist(),
    "flg": (np.random.choice([True, False], size=n_rows)).tolist(),
    ** {
        f"vec{i}": np.random.rand(n_rows) for i in range(200)
    }
}

df = pl.DataFrame(data)


In [3]:
durations = []

for _ in range(200):
    target_ids = np.random.choice(n_rows, size=10).tolist()
    exclude_cd = np.random.choice(n_rows, size=10).tolist()
    threshold = np.random.choice(100)
    target_category = np.random.choice(["A", "B", "C", "D"], size=2)

    start_at = time.time()
    df.filter(
        pl.col("id").is_in(target_ids)
        & ~pl.col("other_cd").is_in(exclude_cd)
        & (pl.col("value") >= threshold)
        & pl.col("category").is_in(target_category)
        & pl.col("flg")
    )
    duration = time.time() - start_at
    durations.append(duration)

In [4]:
filename = f"duration_pl_v{pl.__version__}.pkl"

with open(filename, "wb") as f:
    pickle.dump(durations, f)